In [ ]:
#IMPORTS

import pandas as pd
import numpy as np
import xarray as xr
from google.colab import files


: 

In [ ]:
#COMBINING FILES


#Get number of input files
num_files = int(input("Enter # of files: "))

xarrs = []
for i in range(num_files):
    path = input(f"Path to CSV #{i + 1}: ")
    base = os.path.basename(path)
    name, ext = os.path.splitext(base)
    
    #"500_0.0019" → ["500", "0.0019"]
    try:
        wl_str, bs_str = name.split('_')
        wavelength = float(wl_str)
        beam_size  = float(bs_str)
    except ValueError:
        raise ValueError("Name must be in '<wavelength>_<beam_size>.csv' format")
    
    df  = pd.read_csv(path, header=None)
    arr = df.values.reshape((256, 512, 256))
    
    #Creating xarray
    da = xr.DataArray(
        arr,
        dims=("x", "y", "z"),
        coords={
            "x": np.arange(arr.shape[0]),
            "y": np.arange(arr.shape[1]),
            "z": np.arange(arr.shape[2]),
        },
        attrs={"units": "mW/mm²"}
    ).expand_dims(wavelength=[wavelength]) \
     .assign_coords(beam_size=beam_size)
    
    xarrs.append(da)

combined = xr.concat(xarrs, dim="wavelength").sortby("wavelength")

print(combined)
combined.to_netcdf("combined.nc")

In [ ]:
#INTERPOLATION

#Get user input for uinterpolation
w0 = float(input("Interp start wavelength: "))
w1 = float(input("Interp end wavelength: "))
wNum = int(input("# of wavelength points: "))

b0 = float(input("Interp start beam_size: "))
b1 = float(input("Interp end beam_size: "))
bNum= int(input("# of beam_size points: "))

#Checking inputted wavelength/beam sizes are valid based on inputted files
min_wl = float(combined.wavelength.min())
max_wl = float(combined.wavelength.max())
if w0 < min_wl or w1 > max_wl or w0 > w1:
    raise ValueError("Invalid wavelength inputted, must between min-max of inputted .csv")

bs_vals = combined.coords['beam_size'].values
min_bs = float(np.min(bs_vals))
max_bs = float(np.max(bs_vals))
if b0 < min_bs or b1 > max_bs or b0 > b1:
    raise ValueError("Invalid beam size inputted, must between min-max of inputted .csv")

waveList = np.linspace(w0, w1, wNum) 
beamList = np.linspace(b0, b1, bNum)  

#Interpolate and save as new file
if (bNum == 1):
    fluence_interp = combined.interp(wavelength= waveList, method = "linear")
elif (wNum == 1):
    fluence_interp = combined.interp(beam_size = beamList, method = "linear")
else:
    fluence_interp = combined.interp(wavelength = waveList, beam_size = beamList, method = "linear")

#Final result
print(fluence_interp)
fluence_interp.to_netcdf("fluence_interp.nc")
